In [1]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
import pickle 
import os
from datetime import date
from keras.preprocessing import image
model=load_model('facefeatures_new_model.h5')

Using TensorFlow backend.
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
with open('labels.pickle','rb') as f:
    n_dict=pickle.load(f)

In [3]:
att_dict={}
rev_dict={}
rev_dict={k:v for v,k in n_dict.items()}

In [4]:
rev_dict

{0: 'kushagra', 1: 'priyanshu', 2: 'rahul', 3: 'vishal_gorai'}

In [5]:
roll_dict={'kushagra':'10111','priyanshu':'10377','rahul':'10229','vishal_gorai':'10385'}

In [6]:
face_cascade=cv2.CascadeClassifier('./haar_cascades/data/haarcascade_frontalface_default.xml')

In [7]:
def face_extractor(img):
    faces=face_cascade.detectMultiScale(img,scaleFactor=1.3,minNeighbors=5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_image=img[y:y+h,x:x+w]
        
    return cropped_image

In [8]:
def attendance(name,att_dict):
   
    if name not in att_dict:
        att_dict[name]=1
    ##time1=(int)((time.split(":")[1]).split(".")[0])
    writefile(att_dict)
        
    ##if time%60==0:
        ##writefile(att_dict)
        ##att_dict={}
        
    

In [9]:
def writefile(att_dict):
    f=open('attendance.txt','w+')
    for k,v in att_dict.items():
        roll=roll_dict[k]
        f.write("%s\r\n"%roll)
    
    f.close()
        
        
        

In [10]:

video_capture=cv2.VideoCapture(0)
while(True):
    _,frame=video_capture.read()
    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face=cv2.resize(face,(224,224))
        im=Image.fromarray(face,'RGB')
        
        img_array=np.array(im)
        img_array=np.expand_dims(img_array,axis=0)
        pred=model.predict(img_array)
        #print(pred)
        id_=np.argmax(pred[0])
        name=rev_dict[id_]  
        attendance(name,att_dict)
        cv2.putText(frame,name,(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    
    cv2.imshow('video',frame)
    if cv2.waitKey(1) &0xFF==ord('q'):
        break
        
        
video_capture.release()
cv2.destroyAllWindows()
        
        
        
        